In [1]:
import firebase_admin
from firebase_admin import credentials
from firebase_admin import db
from firebase_admin import storage


cred = credentials.Certificate("admin-sdk.json")
firebase_admin.initialize_app(cred,{
    'databaseURL': 'https://iot-solar-database-default-rtdb.firebaseio.com/',
    'storageBucket':'iot-solar-database.appspot.com'
})

bucket = storage.bucket()

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

# Make numpy printouts easier to read.
np.set_printoptions(precision=3, suppress=True)

In [ ]:
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental import preprocessing

print(tf.__version__)

In [ ]:
raw_dataset = pd.read_csv('dataset-solar\plant1_S+W_float.csv')

In [ ]:
dataset = raw_dataset.copy()
dataset.tail()

In [ ]:
dataset['HOUR'],dataset['MINUTE']=dataset['TIME'].str.split(':',1).str
del dataset['TIME']
dataset['DAY'],dataset['MONTH_YEAR']=dataset['DATE'].str.split('/',1).str
del dataset['DATE']
dataset['MONTH'],dataset['YEAR']=dataset['MONTH_YEAR'].str.split('/',1).str
del dataset['MONTH_YEAR']
dataset.head()

In [ ]:
del dataset['YEAR']
dataset = dataset[ ['MINUTE'] + [ col for col in dataset.columns if col != 'MINUTE' ] ]
dataset = dataset[ ['HOUR'] + [ col for col in dataset.columns if col != 'HOUR' ] ]
dataset = dataset[ ['MONTH'] + [ col for col in dataset.columns if col != 'MONTH' ] ]
dataset = dataset[ ['DAY'] + [ col for col in dataset.columns if col != 'DAY' ] ]
del dataset['DAY']
del dataset['MONTH']
dataset

In [ ]:
dataset['HOUR'] = dataset['HOUR'].apply(lambda x: float(x))
dataset['MINUTE'] = dataset['MINUTE'].apply(lambda x: float(x))
dataset

In [ ]:
#dataset = pd.get_dummies(dataset, prefix='', prefix_sep='')
dataset.tail()

In [ ]:
train_dataset = dataset.sample(frac=0.8, random_state=0)
test_dataset = dataset.drop(train_dataset.index)

In [ ]:
sns.pairplot(train_dataset[['AMBIENT_TEMPERATURE', 'MODULE_TEMPERATURE', 'IRRADIATION', 'DC_POWER', 'AC_POWER']], diag_kind='kde')

In [ ]:
train_dataset.describe().transpose()

In [ ]:
train_features = train_dataset.copy()
test_features = test_dataset.copy()

train_labels = train_features.pop('AC_POWER')
test_labels = test_features.pop('AC_POWER')

In [ ]:
train_dataset.describe().transpose()[['mean', 'std']]

In [ ]:
normalizer = preprocessing.Normalization()

In [ ]:
normalizer.adapt(np.array(train_features))
print(normalizer.mean.numpy())

In [ ]:
first = np.array(train_features[:1])

with np.printoptions(precision=2, suppress=True):
  print('First example:', first)
  print()
  print('Normalized:', normalizer(first).numpy())

In [ ]:
irradiation = np.array(train_features['IRRADIATION'])

irradiation_normalizer = preprocessing.Normalization(input_shape=[1,])
irradiation_normalizer.adapt(irradiation)

In [ ]:
irradiation_model = tf.keras.Sequential([
    irradiation_normalizer,
    layers.Dense(units=1)
])

irradiation_model.summary()

In [ ]:
irradiation_model.predict(irradiation[:10])

In [ ]:
irradiation_model.compile(
    optimizer=tf.optimizers.Adam(learning_rate=0.1),
    loss='mean_absolute_error')

In [ ]:
%%time
history = irradiation_model.fit(
    train_features, train_labels,
    epochs=100,
    # suppress logging
    verbose=0,
    # Calculate validation results on 20% of the training data
    validation_split = 0.2)

In [ ]:
hist = pd.DataFrame(history.history)
hist['epoch'] = history.epoch
hist.tail()

In [ ]:
def plot_loss(history):
  plt.plot(history.history['loss'], label='loss')
  plt.plot(history.history['val_loss'], label='val_loss')
  #plt.ylim([0, 10])
  plt.xlabel('Epoch')
  plt.ylabel('Error [IRRADIATION]')
  plt.legend()
  plt.grid(True)

In [ ]:
plot_loss(history)

In [ ]:
test_results = {}

test_results['irradiation_model'] = irradiation_model.evaluate(
    test_features,
    test_labels, verbose=0)

In [ ]:
x = tf.linspace(0.0, 1.5, 200)
y = irradiation_model.predict(x)

In [ ]:
def plot_irradiation(x, y):
  plt.scatter(train_features['IRRADIATION'], train_labels, label='Data')
  plt.plot(x, y, color='k', label='Predictions')
  plt.xlabel('IRRADIATION')
  plt.ylabel('AC power')
  plt.legend()

In [ ]:
plot_irradiation(x,y)

In [ ]:
linear_model = tf.keras.Sequential([
    normalizer,
    layers.Dense(units=1)
])

In [ ]:
linear_model.predict(train_features[:10])

In [ ]:
linear_model.layers[1].kernel

In [ ]:
linear_model.compile(
    optimizer=tf.optimizers.Adam(learning_rate=0.1),
    loss='mean_absolute_error')

In [ ]:
%%time
history = linear_model.fit(
    train_features, train_labels, 
    epochs=100,
    # suppress logging
    verbose=0,
    # Calculate validation results on 20% of the training data
    validation_split = 0.2)

In [ ]:
plot_loss(history)

In [ ]:
test_results['linear_model'] = linear_model.evaluate(
    test_features, test_labels, verbose=0)

In [ ]:
def build_and_compile_model(norm):
  model = keras.Sequential([
      norm,
      layers.Dense(64, activation='relu'),
      layers.Dense(64, activation='relu'),
      layers.Dense(1)
  ])

  model.compile(loss='mean_absolute_error',
                optimizer=tf.keras.optimizers.Adam(0.001))
  return model

In [ ]:
dnn_irradiation_model = build_and_compile_model(irradiation_normalizer)
dnn_irradiation_model.summary()

In [ ]:
%%time
history = dnn_irradiation_model.fit(
    train_features, train_labels,
    validation_split=0.2,
    verbose=0, epochs=100)

In [ ]:
plot_loss(history)

In [ ]:
y = dnn_irradiation_model.predict(x)

In [ ]:
plot_irradiation(x, y)

In [ ]:
test_results['dnn_irradiation_model'] = dnn_irradiation_model.evaluate(
    test_features, test_labels,
    verbose=0)

In [ ]:
dnn_model = build_and_compile_model(normalizer)
dnn_model.summary()

In [ ]:
%%time
history = dnn_model.fit(
    train_features, train_labels,
    validation_split=0.2,
    verbose=0, epochs=100)

In [ ]:
plot_loss(history)

In [ ]:
test_results['dnn_model'] = dnn_model.evaluate(test_features, test_labels, verbose=0)

In [ ]:
uio = test_features[test_features['IRRADIATION']>1]
print(uio)
print()
print(dnn_model.predict(uio).flatten())

In [ ]:
pd.DataFrame(test_results, index=['Mean absolute error [AC_POWER]']).T

In [ ]:
test_predictions = dnn_model.predict(test_features).flatten()

a = plt.axes(aspect='equal')
plt.scatter(test_labels, test_predictions)
plt.xlabel('True Values [AC_POWER]')
plt.ylabel('Predictions [AC_POWER]')
lims = [-100, 1400]
plt.xlim(lims)
plt.ylim(lims)
_ = plt.plot(lims, lims)


In [ ]:
error = test_predictions - test_labels
plt.hist(error, bins=25)
plt.xlabel('Prediction Error [AC_POWER]')
_ = plt.ylabel('Count')

In [ ]:
dnn_model.save('dnn_model')

In [ ]:
reloaded = tf.keras.models.load_model('dnn_model')

test_results['reloaded'] = reloaded.evaluate(
    test_features, test_labels, verbose=0)

In [ ]:
pd.DataFrame(test_results, index=['Mean absolute error [AC_POWER]']).T